# Семинар 1. Предобработка текстовых данных

В начале любой работы с текстом, как правило, требуется выполнить одни и теже действия: удалить все лишнее, разбить на предложения, токенизировать, нормализовать. На этом занятии мы разберем каждый из этих этапов. 

In [1]:
# сразу импортируем все нужные библиотеки
import string
from rusenttokenize import ru_sent_tokenize
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
from nltk import sent_tokenize
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from pymorphy2 import MorphAnalyzer
from pymystem3 import Mystem
from nltk.corpus import stopwords
from string import punctuation
import re, os, json
mystem = Mystem(entire_input=False)
morph = MorphAnalyzer()
# если есть ошибки, доустановите библиотеки

## Удаление лишнего

Часто в данных, с которыми нам нужно работать помимо текста присутствует ещё какая-то лишняя информация: тэги, ссылки, код, разметка. Она, конечно, не всегда лишняя, но обычно от неё лучше избавиться.

Возьмем в качестве примера статью с Хабра (она была в презе). Она скачана автоматически и в там остались некоторые тэги.

In [4]:
text = '''
<div xmlns="http://www.w3.org/1999/xhtml">Привет! Меня зовут Денис Кирьянов, я работаю в Сбербанке и занимаюсь проблемами обработки естественного языка (NLP). Однажды нам понадобилось выбрать синтаксический парсер для работы с русским языком. Для этого мы углубились в дебри морфологии и токенизации, протестировали разные варианты и оценили их применение. Делимся опытом в этом посте.<br/>
<br/><img data-src="https://habrastorage.org/getpro/habr/post_images/c87/ec8/f26/c87ec8f26a969cf54915271e24abcba1.png" src="/img/image-loader.svg"/><br/>
<a name="habracut"></a><br/><h2>Подготовка к отбору.</h2><br/>Начнём с основ: как все работает?<br/>Мы берем текст, проводим токенизацию и получаем некоторый массив псевдослов-токенов. Этапы дальнейшего анализа укладываются в пирамиду:<br/>
<br/><img data-src="https://habrastorage.org/getpro/habr/post_images/b2f/cd9/0aa/b2fcd90aaf42d1eee5ed3ee84fcf27fd.png" src="/img/image-loader.svg"/><br/><br/>Начинается все с морфологии — с анализа формы слова и его грамматических категорий (род, падеж и т.п.). На морфологии базируется синтаксис — взаимоотношения за рамками одного слова, между словами. Синтаксические парсеры, о которых пойдет речь, анализируют текст и выдают структуру зависимостей слов друг от друга.<br/>
<br/><h3>Грамматика зависимостей и грамматика непосредственных составляющих.</h3><br/>
Есть два основных подхода к синтаксическому анализу, которые в лингвистической теории существуют примерно на равных.<br/>'''

В html все тэги заключаются в угловые скобки. Мы можем испьзовать это, чтобы легко избавиться от всех тэгов сразу. Напишем регулярное выражение, которое будет захватывать всё, что попадает между символами < и >, и не является '>'.

In [5]:
#re - модуль регулярных выражений в питоне
# функция sub заменяет все, что подходит под шаблон, на указанный текст
def remove_tags_1(text):
    return re.sub(r'<[^>]+>', '', text)

Проверим как работает наша функция.

In [6]:
print(remove_tags_1(text))


Привет! Меня зовут Денис Кирьянов, я работаю в Сбербанке и занимаюсь проблемами обработки естественного языка (NLP). Однажды нам понадобилось выбрать синтаксический парсер для работы с русским языком. Для этого мы углубились в дебри морфологии и токенизации, протестировали разные варианты и оценили их применение. Делимся опытом в этом посте.

Подготовка к отбору.Начнём с основ: как все работает?Мы берем текст, проводим токенизацию и получаем некоторый массив псевдослов-токенов. Этапы дальнейшего анализа укладываются в пирамиду:
Начинается все с морфологии — с анализа формы слова и его грамматических категорий (род, падеж и т.п.). На морфологии базируется синтаксис — взаимоотношения за рамками одного слова, между словами. Синтаксические парсеры, о которых пойдет речь, анализируют текст и выдают структуру зависимостей слов друг от друга.
Грамматика зависимостей и грамматика непосредственных составляющих.
Есть два основных подхода к синтаксическому анализу, которые в лингвистической теор

Можно заметить, что в некоторых местах удаление тэгов приводит к тому, что между точкой и началом следующего 
предложения нет пробела, а это может помешать правильно токенизировать текст,
поэтому сделаем так, чтобы тэг заменялся пробелом

In [7]:
def remove_tags_2(text):
    return re.sub(r'<[^>]+>', ' ', text)

In [8]:
print(remove_tags_2(text))


 Привет! Меня зовут Денис Кирьянов, я работаю в Сбербанке и занимаюсь проблемами обработки естественного языка (NLP). Однажды нам понадобилось выбрать синтаксический парсер для работы с русским языком. Для этого мы углубились в дебри морфологии и токенизации, протестировали разные варианты и оценили их применение. Делимся опытом в этом посте. 
   
    Подготовка к отбору.  Начнём с основ: как все работает? Мы берем текст, проводим токенизацию и получаем некоторый массив псевдослов-токенов. Этапы дальнейшего анализа укладываются в пирамиду: 
    Начинается все с морфологии — с анализа формы слова и его грамматических категорий (род, падеж и т.п.). На морфологии базируется синтаксис — взаимоотношения за рамками одного слова, между словами. Синтаксические парсеры, о которых пойдет речь, анализируют текст и выдают структуру зависимостей слов друг от друга. 
  Грамматика зависимостей и грамматика непосредственных составляющих.  
Есть два основных подхода к синтаксическому анализу, которые 

Теперь куски текста не слипаются, но появились последовательности из нескольких пробелов, чтобы убрать их добавим ещё одно регулярное выражение и применим его к тексту, из которого уже удалили тэги

In [9]:
def remove_tags_3(text):
    no_tags_text = re.sub(r'<[^>]+>', ' ', text)
    no_space_sequences_text = re.sub('  +', ' ', no_tags_text)
    return no_space_sequences_text

In [10]:
print(remove_tags_3(text))


 Привет! Меня зовут Денис Кирьянов, я работаю в Сбербанке и занимаюсь проблемами обработки естественного языка (NLP). Однажды нам понадобилось выбрать синтаксический парсер для работы с русским языком. Для этого мы углубились в дебри морфологии и токенизации, протестировали разные варианты и оценили их применение. Делимся опытом в этом посте. 
 
 Подготовка к отбору. Начнём с основ: как все работает? Мы берем текст, проводим токенизацию и получаем некоторый массив псевдослов-токенов. Этапы дальнейшего анализа укладываются в пирамиду: 
 Начинается все с морфологии — с анализа формы слова и его грамматических категорий (род, падеж и т.п.). На морфологии базируется синтаксис — взаимоотношения за рамками одного слова, между словами. Синтаксические парсеры, о которых пойдет речь, анализируют текст и выдают структуру зависимостей слов друг от друга. 
 Грамматика зависимостей и грамматика непосредственных составляющих. 
Есть два основных подхода к синтаксическому анализу, которые в лингвисти

Теперь текст более менее чистый. Присвоим его в переменную text

In [11]:
text = remove_tags_3(text)

Очистка текста достаточно индивидуальна и каждый раз приходится писать какие-то новые правила. А вот разбиение на предложения, токенизация и лемматизация - на 90% универсальны (только если вы не работаете с соцсетями!:)), т.е. одни и теже решения будут работать во всех задачах. К тому же, есть уже хорошие готовые инструменты для всего этого. 

## Сегментация (разбиение на предложения)

В __nltk__ есть уже готовая функция для разбивки на предложения. (Тут у нас очень просто пример)

In [12]:
sent_tokenize(text, 'russian')[:10]

['\n Привет!',
 'Меня зовут Денис Кирьянов, я работаю в Сбербанке и занимаюсь проблемами обработки естественного языка (NLP).',
 'Однажды нам понадобилось выбрать синтаксический парсер для работы с русским языком.',
 'Для этого мы углубились в дебри морфологии и токенизации, протестировали разные варианты и оценили их применение.',
 'Делимся опытом в этом посте.',
 'Подготовка к отбору.',
 'Начнём с основ: как все работает?',
 'Мы берем текст, проводим токенизацию и получаем некоторый массив псевдослов-токенов.',
 'Этапы дальнейшего анализа укладываются в пирамиду: \n Начинается все с морфологии — с анализа формы слова и его грамматических категорий (род, падеж и т.п.).',
 'На морфологии базируется синтаксис — взаимоотношения за рамками одного слова, между словами.']

Но вы можете придумать и потестить всякие примерчики, которые осложнят жизнь сегментатору. К вопросу о том, как важно понимать, с какими данными вы работаете.

In [13]:
small_text = "илок. сила"
sent_tokenize(small_text, 'russian')

['илок.', 'сила']

### Запомните: программисты (nlp-инженеры, называйте их как хотите) не любят смотреть в данные. Их больше интересуют их модельки. Смотреть в данные - ВАША задача. Хорошо знайте ваши данные и будет вам КЛ-счастье

У DeepPavlov есть библиотека [**rusenttokenizer**](https://github.com/deepmipt/ru_sentence_tokenizer). (У меня всегда вылезают варнинги, и не только у меня)

In [14]:
ru_sent_tokenize(text)[:10]

['Привет!',
 'Меня зовут Денис Кирьянов, я работаю в Сбербанке и занимаюсь проблемами обработки естественного языка (NLP).',
 'Однажды нам понадобилось выбрать синтаксический парсер для работы с русским языком.',
 'Для этого мы углубились в дебри морфологии и токенизации, протестировали разные варианты и оценили их применение.',
 'Делимся опытом в этом посте.',
 'Подготовка к отбору.',
 'Начнём с основ: как все работает?',
 'Мы берем текст, проводим токенизацию и получаем некоторый массив псевдослов-токенов.',
 'Этапы дальнейшего анализа укладываются в пирамиду: \n Начинается все с морфологии — с анализа формы слова и его грамматических категорий (род, падеж и т.п.).',
 'На морфологии базируется синтаксис — взаимоотношения за рамками одного слова, между словами.']

In [16]:
ru_sent_tokenize('- Куда? - говорю, - Куда мы едем, батя?')

['- Куда? - говорю, - Куда мы едем, батя?']

В проекте Natasha есть библиотека [**razdel**](https://github.com/natasha/razdel). Она чуть более навороченная. Установите Наташу у себя и погоняйте сами, у меня была измененная версия, которую допиливала я.

In [17]:
sents = list(sentenize('БЕБИ.РУ'))

In [18]:
# числа тут - это спаны, индексы начала и конца предложения в изначальном тексте
sents[:10]

[Substring(0, 7, 'БЕБИ.РУ')]

In [19]:
# у объекта Substring есть атрибуты start, stop и text. С помощью них можно вытащить нужное
[sent.text for sent in sents]

['БЕБИ.РУ']

Если все-таки нужно добавить каких-то специфичных правил разбиения на предложения, можно опять же восползоваться регулярными выражениями. Однако в этом случае регулярка будет посложнее. 

## Токенизация

Мы разбили текст на предложения. Теперь предложения нужно разбить на токены. Под токенами обычно понимаются слова (псевдослова), но это могут быть и какие-то более длинные или короткие куски. 

Самый простой способ токенизации -- стандартный питоновский __str.split__ метод.
    По умолчанию он разбивает текст по последовательностям пробелом 

In [20]:
text.split()[:20]

['Привет!',
 'Меня',
 'зовут',
 'Денис',
 'Кирьянов,',
 'я',
 'работаю',
 'в',
 'Сбербанке',
 'и',
 'занимаюсь',
 'проблемами',
 'обработки',
 'естественного',
 'языка',
 '(NLP).',
 'Однажды',
 'нам',
 'понадобилось',
 'выбрать']

Большая часть слов отделяется, но знаки препинания лепятся к словам.
Можно пройтись по всем словам и убрать из них пунктцацию с методом str.strip. Помините, что в каких-то задачах нам нужны знаки препинания, в каких-то нет.

In [22]:
#основные знаки преминания хранятся в питоновском модуле string.punctuation
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [23]:
# в этом списке не хватает кавычек-ёлочек, лапок, длинного тире и многоточия (из самых стандартных вещей)
string.punctuation += '«»—…“”'

In [24]:
[word.strip(string.punctuation) for word in text.split()][:10]

['Привет',
 'Меня',
 'зовут',
 'Денис',
 'Кирьянов',
 'я',
 'работаю',
 'в',
 'Сбербанке',
 'и']

Так не будут удаляться дефисы и точке в сокращениях, не разделенных пробелом.

In [25]:
'как-нибудь'.strip(punctuation)

'как-нибудь'

In [26]:
'т.е.'.strip(punctuation)

'т.е'

Ещё слова можно извлечь с помощью простого регулярного выражения:

In [27]:
re.findall('\w+', text)[:10]

['Привет',
 'Меня',
 'зовут',
 'Денис',
 'Кирьянов',
 'я',
 'работаю',
 'в',
 'Сбербанке',
 'и']

Ещё есть готовые токенизаторы из nltk. Они не удаляют пунктуацию, а выделяют её отдельным токеном.

Например **wordpunct_tokenizer** разбирает по регулярке - *'\w+|[^\w\s]+'* 

In [28]:
wordpunct_tokenize(text)[:10]

['Привет', '!', 'Меня', 'зовут', 'Денис', 'Кирьянов', ',', 'я', 'работаю', 'в']

Ещё есть **word_tokenize**. Он также построен на регулярках, но они там более сложные (учитывается последовательность некоторых 
символов, символы начала, конца слова и т.д). 

Специально подобранного под русский язык токенизатора там нет, 
но и с английским всё работает достаточно хорошо --
сокращения типа т.к собираются в один токен, дефисные слова тоже не разделяются, многоточия тут тоже не отделяются, но это можно поправить.

In [29]:
word_tokenize(text)[:10]

['Привет', '!', 'Меня', 'зовут', 'Денис', 'Кирьянов', ',', 'я', 'работаю', 'в']

In [30]:
word_tokenize('vk.ru')[:10]

['vk.ru']

И в razdel тоже есть токенизация

In [31]:
[token.text for token in list(razdel_tokenize(text))[:10]]

['Привет', '!', 'Меня', 'зовут', 'Денис', 'Кирьянов', ',', 'я', 'работаю', 'в']

Как и всегда, думайте, что важно для вашей задачи и смотрите в данные.

# Нормализация

Полученные токены тоже чаще всего нужно привести к какому-то стандартному виду.

Самое простое и очевидное - привести всё к одному регистру:

In [32]:
'Слово'.lower()
# если не нужно разбивать на предложения, то это можно сделать в самом начале
# text.lower()

'слово'

Для языков со слабым словоизменением этого может быть достаточно. Но для флективного русского лучше использовать стемминг или лемматизацию.

## Стемминг

Стемминг - это урезание слова до его "основы" (стема), т.е. такой части, которая является общей для всех словоформ в парадигме слова. По крайней мере так в теории. На практике стемминг сводится к отбрасыванию частотных окончаний.

Самый известный стеммер - стеммер Портера (или snowball стеммер). 
Подробнее про стеммер Портера можно почитать вот тут - <https://medium.com/@eigenein/стеммер-портера-для-русского-языка-d41c38b2d340>  
А совсем подробнее вот тут - <http://snowball.tartarus.org/algorithms/russian/stemmer.html>  
Почему он так называется? Так назывался язык программирования, который Портер написал для стеммеров. Язык так называется в созвучие языку SNOBOL. Вот комментарий самого Портера:

`Since it effectively provides a ‘suffix STRIPPER GRAMmar’, I had toyed with the idea of calling it ‘strippergram’, but good sense has prevailed, and so it is ‘Snowball’ named as a tribute to SNOBOL, the excellent string handling language of Messrs Farber, Griswold, Poage and Polonsky from the 1960s.`

Готовые стеммеры для разных языков есть в nltk. Работают они вот так:

In [33]:
from nltk.stem.snowball import SnowballStemmer

In [35]:
stemmer = SnowballStemmer('russian')

In [36]:
[(word, stemmer.stem(word)) for word in word_tokenize(text)][:30]

[('Привет', 'привет'),
 ('!', '!'),
 ('Меня', 'мен'),
 ('зовут', 'зовут'),
 ('Денис', 'денис'),
 ('Кирьянов', 'кирьян'),
 (',', ','),
 ('я', 'я'),
 ('работаю', 'работа'),
 ('в', 'в'),
 ('Сбербанке', 'сбербанк'),
 ('и', 'и'),
 ('занимаюсь', 'занима'),
 ('проблемами', 'проблем'),
 ('обработки', 'обработк'),
 ('естественного', 'естествен'),
 ('языка', 'язык'),
 ('(', '('),
 ('NLP', 'NLP'),
 (')', ')'),
 ('.', '.'),
 ('Однажды', 'однажд'),
 ('нам', 'нам'),
 ('понадобилось', 'понадоб'),
 ('выбрать', 'выбра'),
 ('синтаксический', 'синтаксическ'),
 ('парсер', 'парсер'),
 ('для', 'для'),
 ('работы', 'работ'),
 ('с', 'с')]

Недостатки стемминга достаточно очевидные:  
1) с супплетивными формами или редкими окончаниями слова стемминг работать не умеет  
2) к одной основе могут приводится разные слова  
3) к разным основам могут сводиться формы одного слова  
4) приставки не отбрасываются (иногда это нужно)
5) Иногда получается бред типа "меня" -> "мен"

Поковыряйтесь со стеммером для английского.

In [37]:
stemmer_en = SnowballStemmer('english')

In [38]:
stemmer_en.stem('are')

'are'

In [39]:
stemmer_en.stem('smarter')

'smarter'

In [41]:
stemmer_en.stem('meeting') # а если это сущ. "встреча"?

'meet'

In [44]:
stemmer.stem("курить")

'кур'

In [45]:
stemmer.stem("куры")

'кур'

# Лемматизация

Лемматизация - это замена словоформы слова в парадигме на лемму. 



Например, для разных форм глагола леммой обычно является неопределенная форма, а для существительного форма мужского рода единственного числа. Это позволяет избавиться от недостатков стемминга (будет, был - одна лемма), (пролить, пролом - разные). Однако лемматизация значительно сложнее. 

К счастью есть готовые (хорошие?) лемматизаторы. Для русского основых варианта два: Mystem и Pymorphy. (Естественно, это уже старые технологии, они довольно плохо учитывают контекст, можете сами потестить разные штуки с омонимией).


### Mystem


Майстем работает немного лучше и сам токенизирует,
поэтому можно в него засовывать сырой текст. НО! Работает медленно

In [50]:
# mystem.lemmatize функция лемматизации в майстеме
# сам объект mystem нужно заранее инициализировать
# мы сделали это в начале тетрадки строчкой "mystem = Mystem(entire_input=False)"
# entire_input=False удаляет пробелы и переносы строк из выдачи
mystem.lemmatize('Все эти типы стали есть в нашем цехе')

['весь', 'этот', 'тип', 'становиться', 'быть', 'в', 'наш', 'цех']

Недостатки Mystem: это продукт Яндекса с некоторыми ограничениями на использование, больше он не развивается.

Важным достоинством Mystem является то, что он работает не с отдельными словами, а с целым предложением. При определении нужной леммы учитывается контекст, что позволяет во многих случаях разрешать омонимию.

### Pymorphy

Pymorphy - открытый и развивается (можно поучаствовать на гитхабе)

Ссылка на репозиторий: https://github.com/kmike/pymorphy2

У него нет встроенной токенизации и он расценивает всё как слово. Когда есть несколько вариантов, он выдает их с вероятностостями, которые расчитатны на корпусе со снятой неоднозначностью (OpenCorpora, насколько я знаю). Это лучше стемминга, но хуже майстема.

In [52]:
# основной метод - pymorphy.parse
words_analized = [morph.parse(token) for token in word_tokenize('Все эти типы стали есть в нашем цехе')]
words_analized


[[Parse(word='все', tag=OpencorporaTag('ADJF,Apro plur,nomn'), normal_form='весь', score=0.357142, methods_stack=((<DictionaryAnalyzer>, 'все', 703, 20),)),
  Parse(word='всё', tag=OpencorporaTag('ADJF,Apro neut,sing,nomn'), normal_form='весь', score=0.214285, methods_stack=((<DictionaryAnalyzer>, 'всё', 703, 14),)),
  Parse(word='все', tag=OpencorporaTag('ADJF,Apro inan,plur,accs'), normal_form='весь', score=0.142857, methods_stack=((<DictionaryAnalyzer>, 'все', 703, 23),)),
  Parse(word='всё', tag=OpencorporaTag('ADVB'), normal_form='всё', score=0.142857, methods_stack=((<DictionaryAnalyzer>, 'всё', 3, 0),)),
  Parse(word='всё', tag=OpencorporaTag('ADJF,Apro neut,sing,accs'), normal_form='весь', score=0.142857, methods_stack=((<DictionaryAnalyzer>, 'всё', 703, 17),))],
 [Parse(word='эти', tag=OpencorporaTag('ADJF,Apro,Subx,Anph plur,nomn'), normal_form='этот', score=0.571428, methods_stack=((<DictionaryAnalyzer>, 'эти', 3099, 19),)),
  Parse(word='эти', tag=OpencorporaTag('ADJF,Apro,

In [54]:
# Она похожа на analyze в майстеме только возрващает список объектов Parse
# Первый в списке - самый вероятный разбор (у каждого есть score)
# Информация достается через атрибут (Parse.word - например)
# Грамматическая информация хранится в объекте OpencorporaTag и из него удобно доставать
# части речи или другие категории
print('Первое слово - ', words_analized[0][0].word)
print('Разбор первого слова - ', words_analized[0][0])
print('Лемма первого слова - ', words_analized[0][0].normal_form)
print('Грамматическая информация первого слова - ', words_analized[0][0].tag)
print('Часть речи первого слова - ', words_analized[0][0].tag.POS)
print('Род первого слова - ', words_analized[0][0].tag.gender)
print('Число первого слова - ', words_analized[0][0].tag.number)
print('Падеж первого слова - ', words_analized[0][0].tag.case)

Первое слово -  все
Разбор первого слова -  Parse(word='все', tag=OpencorporaTag('ADJF,Apro plur,nomn'), normal_form='весь', score=0.357142, methods_stack=((<DictionaryAnalyzer>, 'все', 703, 20),))
Лемма первого слова -  весь
Грамматическая информация первого слова -  ADJF,Apro plur,nomn
Часть речи первого слова -  ADJF
Род первого слова -  None
Число первого слова -  plur
Падеж первого слова -  nomn


## Дополнительная очистка текста

Пунктуация часто совсем не нужна и поэтому можно выбросить её заранее. Если нужно обрабатывать много текста, это может немного ускорить процесс.


In [7]:
# Оставим только буквенно-численные токены
# Это не самый лучший вариант, так как удалятся сокращения с точкой, слова через дефис
text = 'В этом случае слова вроде т.к. и по-другому не пройдут фильтр и будут удалены.'
good_tokens = [word for word in word_tokenize(text) if word.isalnum()]
[morph.parse(token)[0].normal_form for token in good_tokens]

['в',
 'это',
 'случай',
 'слово',
 'вроде',
 'и',
 'не',
 'пройти',
 'фильтр',
 'и',
 'быть',
 'удалить']

Можно убрать стоп-слова (предлоги, союзы, местоимения, частотные слова). Сам термин стоп-слово происходит из информационного поиска.  
Удаление таких слов позволяло сократить размер текста и не сильно испортить выдачу или даже улучшить её, поднимая релевантность документам со значимыми словами. Со временем от такой практики, в основном, отказались - память стала дешевой (и повились всякие алгоритмы для сокращения потребления памяти), а для учёта значимости придумали такую штуку, как IDF.  

In [2]:
import json
from collections import Counter

with open('bmr_normed.json') as json_file:
    bmr = json.load(json_file)
    bmr_toked = []
    for i, post in enumerate(bmr):
        bmr_toked.extend([token.text for token in list(razdel_tokenize(post['text']))])
        
    cnt = Counter(bmr_toked)

In [4]:
print(*cnt.most_common(20), sep='\n')

(',', 29521)
('.', 18754)
('и', 8979)
('в', 7111)
('—', 6081)
('не', 5202)
('"', 5083)
(')', 4123)
('...', 4032)
('на', 3847)
('!', 3351)
('что', 2884)
('с', 2853)
(':', 2147)
('(', 1837)
('я', 1813)
('?', 1710)
('как', 1691)
('И', 1665)
('_', 1519)


Смотрите, самыми частотными оказались знаки пунктуации, союзы и предлоги (из-за того, что мы не приводили к нижнему регистру, вылезло даже заглавное И)

Во многих поисковых движках стоп-слова всё ещё используются. Часто их используют и в практических задачах (классификации, тематическом моделировании). 

In [5]:
# стоп-слова есть в nltk
stops = stopwords.words('russian')
print(*stops, sep=', ')

и, в, во, не, что, он, на, я, с, со, как, а, то, все, она, так, его, но, да, ты, к, у, же, вы, за, бы, по, только, ее, мне, было, вот, от, меня, еще, нет, о, из, ему, теперь, когда, даже, ну, вдруг, ли, если, уже, или, ни, быть, был, него, до, вас, нибудь, опять, уж, вам, ведь, там, потом, себя, ничего, ей, может, они, тут, где, есть, надо, ней, для, мы, тебя, их, чем, была, сам, чтоб, без, будто, чего, раз, тоже, себе, под, будет, ж, тогда, кто, этот, того, потому, этого, какой, совсем, ним, здесь, этом, один, почти, мой, тем, чтобы, нее, сейчас, были, куда, зачем, всех, никогда, можно, при, наконец, два, об, другой, хоть, после, над, больше, тот, через, эти, нас, про, всего, них, какая, много, разве, три, эту, моя, впрочем, хорошо, свою, этой, перед, иногда, лучше, чуть, том, нельзя, такой, им, более, всегда, конечно, всю, между


Список не идеальный и его можно расширять под свои задачи.

In [8]:
words_normalized = [morph.parse(token)[0].normal_form for token in good_tokens]
[word for word in words_normalized if word not in stops]

['это', 'случай', 'слово', 'вроде', 'пройти', 'фильтр', 'удалить']

## Предобработка для других языков

Скорее всего вы будете работать в основном только с русским языком. Но знать, чем пользоваться для других на всякий случай - тоже полезно

Nltk по умолчанию адаптирован под английский язык.

Есть еще одна библиотека, про которую стоит рассказать - [**SpaCy**](https://spacy.io/). Это многоцелевая многоязычная библиотека. Если вам понадобится серьезно работать с английским, то лучшим вариантом будет использовать SpaCy. Другие языки там тоже поддерживаются (см. документацию - ), но не настолько хорошо как английский язык. 

В SpaCy много всего и мы будем возвращаться к ней по ходу курса. Пока посмотрим на интрументы базовой предобработки.

In [120]:
!pip install spacy
!python -m spacy download en_core_web_sm
!python -m spacy download de_core_news_sm

You should consider upgrading via the '/Users/marimitchurina/opt/anaconda3/bin/python -m pip install --upgrade pip' command.
^C
     |███████████████████▍            | 9.1 MB 2.6 MB/s eta 0:00:03^C██████████████▊            | 9.2 MB 2.6 MB/s eta 0:00:03
ERROR: Exception:
Traceback (most recent call last):
  File "/Users/marimitchurina/opt/anaconda3/lib/python3.7/site-packages/pip/_internal/cli/base_command.py", line 180, in _main
    status = self.run(options, args)
  File "/Users/marimitchurina/opt/anaconda3/lib/python3.7/site-packages/pip/_internal/cli/req_command.py", line 204, in wrapper
    return func(self, options, args)
  File "/Users/marimitchurina/opt/anaconda3/lib/python3.7/site-packages/pip/_internal/commands/install.py", line 319, in run
    reqs, check_supported_wheels=not options.target_dir
  File "/Users/marimitchurina/opt/anaconda3/lib/python3.7/site-packages/pip/_internal/resolution/resolvelib/resolver.py", line 104, in resolve
    req, requested_extras=()
  File "/Us

In [122]:
# загружаем пайплайн для английского языка
import spacy

nlp = spacy.load("en_core_web_sm")


In [132]:

text = ("Panda eats, shoots and leaves")

In [133]:
doc = nlp(text)

In [134]:
for sent in doc.sents: # достаем предложения
    for token in sent: # достаем токены
        print(token.text, token.lemma_, token.pos_)

Panda panda NOUN
eats eat VERB
, , PUNCT
shoots shoot NOUN
and and CCONJ
leaves leave NOUN


In [87]:
print("Noun phrases:", [chunk.text for chunk in doc.noun_chunks])

Noun phrases: ['the most salient features', 'our culture', 'so much bullshit', 'the opening words', 'the short book', 'Bullshit', 'the philosopher', 'Harry Frankfurt', 'the publication', 'this surprise bestseller', 'the rapid progress', 'research', 'artificial intelligence', 'us', 'our conception', 'bullshit', 'a hallmark', 'human speech', 'troubling implications', 'What', 'philosophical reflections', 'bullshit', 'algorithms', 'it', 'May', 'the company', 'Elon Musk', 'a new language model', 'Transformer', 'It', 'the tech world', 'storm', 'the surface', 'GPT-3', 'a supercharged version', 'the autocomplete feature', 'your smartphone', 'it', 'coherent text', 'an initial input', 'GPT-3’s text-generating abilities', 'anything', 'your phone', 'It', 'pronouns', 'translate', 'some forms', 'common-sense reasoning', 'It', 'fake news articles', 'humans', 'chance', 'a definition', 'it', 'a made-up word', 'a sentence', 'It', 'a paragraph', 'the style', 'a famous author', 'it', 'creative fiction', '

In [88]:
# загружаем пайплайн для немецкого языка
import spacy

nlp = spacy.load("de_core_news_sm")


In [89]:

text = ("Vor den Stadien habe ich bis jetzt zum Glück noch keine wüsten Szenen gesehen."
        "Vorstandschef Timotheus Höttges habe sich ausgesprochen optimistisch gezeigt, schrieb Analyst Robert Grindle in einer Studie vom Montag. "
        "Während der dortigen Räterepublik war er nach dem Krieg in Künstlergruppen und Ausschüssen aktiv."
        "Welches Ergebnis die Diskussion auf EU-Ebene auch letztlich bringt, wichtig ist, dass die Preisentwicklung für die"
        "Menschen verträglicher gestaltet wird“, so Gusenbauer."
        "Weitere Informationen unter www.schnippenburg.de sowie www.eisenzeithaus.de. Es gibt neue Nachrichten auf noz.de!" 
        "Jetzt die Startseite neu laden."
        "Der Initiative 'Zivilcourage', die sich jahrelang für das Denkmal in Form eines offenen " 
        "Der islamistischen Szene Thüringens wurden nach Angaben des Thüringer Innenministeriums "
        "zuletzt etwa 125 Personen zugerechnet, der salafistischen Szene etwa 75 Personen."
        "Allerdings bestand er die EMV-Prüfung nicht, weil er Radios und DVB-T-Empfänger stört."
        )

In [90]:
doc = nlp(text)

In [91]:
for sent in doc.sents: # достаем предложения
    for token in sent: # достаем токены
        print(token.text, token.lemma_, token.pos_)

Vor Vor ADP
den der DET
Stadien Stadium NOUN
habe habe AUX
ich ich PRON
bis bis ADP
jetzt jetzt ADV
zum zum ADP
Glück Glück NOUN
noch noch ADV
keine kein DET
wüsten wüst ADJ
Szenen Szene NOUN
gesehen sehen VERB
. . PUNCT
Vorstandschef Vorstandschef NOUN
Timotheus Timotheus PROPN
Höttges Höttges NOUN
habe habe AUX
sich sich PRON
ausgesprochen aussprechen ADJ
optimistisch optimistisch ADJ
gezeigt zeigen VERB
, , PUNCT
schrieb schreiben VERB
Analyst Analyst NOUN
Robert Robert PROPN
Grindle Grindle PROPN
in in ADP
einer einer DET
Studie Studie NOUN
vom vom ADP
Montag Montag NOUN
. . PUNCT
Während während ADP
der der DET
dortigen dortig ADJ
Räterepublik Räterepublik NOUN
war sein AUX
er ich PRON
nach nach ADP
dem der DET
Krieg Krieg NOUN
in in ADP
Künstlergruppen Künstlergruppen NOUN
und und CCONJ
Ausschüssen Ausschuß NOUN
aktiv aktiv ADJ
. . PUNCT
Welches welch DET
Ergebnis Ergebnis NOUN
die der DET
Diskussion Diskussion NOUN
auf auf ADP
EU-Ebene EU-Ebene NOUN
auch auch ADV
letztlich letzt

In [92]:
print("Noun phrases:", [chunk.text for chunk in doc.noun_chunks])

Noun phrases: ['den Stadien', 'ich', 'Glück', 'noch keine wüsten Szenen', 'Vorstandschef Timotheus Höttges', 'sich', 'Analyst Robert Grindle', 'einer Studie', 'Montag', 'der dortigen Räterepublik', 'er', 'dem Krieg', 'Künstlergruppen', 'Ausschüssen', 'Welches Ergebnis', 'die Diskussion', 'EU-Ebene', 'die Preisentwicklung', 'dieMenschen', 'Welches Ergebnis die Diskussion auf EU-Ebene auch letztlich bringt, wichtig ist, dass die Preisentwicklung für dieMenschen verträglicher gestaltet wird“, so Gusenbauer', 'Weitere Informationen', 'www.schnippenburg.de', 'www.eisenzeithaus.de', 'neue Nachrichten', 'noz.de', 'die Startseite', "Der Initiative 'Zivilcourage", 'die', 'sich', 'das Denkmal', 'Form', 'eines offenen Der islamistischen Szene', 'Thüringens', 'Angaben', 'des Thüringer Innenministeriums', 'etwa 125 Personen', 'der salafistischen Szene', 'er', 'die EMV-Prüfung', 'er', 'Radios', 'DVB-T-Empfänger']


Есть русскоязычный форк spacy - https://github.com/buriy/spacy-ru Но это конечно не полноценный spacy. 